<a href="https://www.kaggle.com/code/aastha0924/forecasting?scriptVersionId=258309154" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# LSTM

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from datetime import datetime
from keras.layers import TimeDistributed, LSTM, Dense, Dropout


2025-08-26 14:45:47.475057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756219547.688331      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756219547.753708      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/forecastingdata/Data1.csv')

# Sort and filter
df.sort_values('PERIOD_START_TIME', inplace=True)
cutoff_date = pd.Timestamp("2022-09-27")

In [3]:
df.head()

,PERIOD_START_TIME,pl,pd,pdv
0,2022-07-04 06:00:00,0.003754,0.975612,0.018548
1,2022-07-04 06:15:00,0.003019,0.976934,0.019894
2,2022-07-04 06:30:00,0.005802,0.977411,0.020060
3,2022-07-04 06:45:00,0.004035,0.977569,0.021209
4,2022-07-04 07:00:00,0.005641,0.981013,0.024581


In [4]:
df.tail()

,PERIOD_START_TIME,pl,pd,pdv
5888,2022-10-01 22:45:00,0.004487,0.744733,0.044438
5889,2022-10-01 23:00:00,0.001543,0.732024,0.031747
5890,2022-10-01 23:15:00,0.001963,0.728793,0.028551
5891,2022-10-01 23:30:00,0.008098,0.738851,0.038595
5892,2022-10-01 23:45:00,0.006501,0.734670,0.036419


In [5]:
# Features
features = ['pl', 'pd', 'pdv']
sequence_length = 5

# --- Normalize each feature ---
scalers = {}
scaled_train = []

for f in features:
    scaler = MinMaxScaler()
    scaled_col = scaler.fit_transform(df[[f]])
    scalers[f] = scaler
    scaled_train.append(scaled_col)

In [6]:
# Combine all scaled features
scaled_train = np.hstack(scaled_train)
# --- Create sequences ---
X, y = [], []
for i in range(len(scaled_train) - sequence_length):
    end_idx = i + sequence_length
    X.append(scaled_train[i:end_idx])
    y.append(scaled_train[end_idx])  # Next time step prediction

X, y = np.array(X), np.array(y)

In [7]:
print("PL Min:", df['pl'].min())
print("PL Max:", df['pl'].max())


PL Min: 0.0005765917323529
PL Max: 0.2220553330366414


In [8]:
# --- Build and train LSTM model ---
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    Dense(32, activation='relu'),
    Dense(16, activation = 'relu'),
    Dense(3)  # Output: pl, pd, pdv
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=20, batch_size=32, verbose=1)


Epoch 1/20


2025-08-26 14:46:02.215568: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


184/184 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0233
Epoch 2/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0036
Epoch 3/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0035
Epoch 4/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0029
Epoch 5/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0029
Epoch 6/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0030
Epoch 7/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0027
Epoch 8/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0027
Epoch 9/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0027
Epoch 10/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0028
Epoch 11/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0029
Epoch 12/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0027
Epoch 13/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0027
Epoch 14/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0026
Epoch 15/20
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0026


In [9]:
# --- Prepare for prediction on 25 Sep 2022 ---
df['PERIOD_START_TIME'] = pd.to_datetime(df['PERIOD_START_TIME'])
df_test = df[df['PERIOD_START_TIME'].dt.date == cutoff_date.date()].copy()
forecast_times = df_test['PERIOD_START_TIME'].values

In [10]:
# Start with last known sequence before 25 Sep
last_sequence = scaled_train[-sequence_length:]
results = []

for i in range(len(forecast_times)):
    input_seq = np.expand_dims(last_sequence, axis=0)  # shape: (1, 5, 3)
    pred_scaled = model.predict(input_seq, verbose=0)[0]  # shape: (3,)

    # Inverse transform per feature
    pred = np.zeros(3)
    for j, f in enumerate(features):
         pred_value = pred_scaled[j].reshape(1, -1)
         pred[j] = scalers[f].inverse_transform(pred_value)[0][0]



    results.append({
        "PERIOD_START_TIME": forecast_times[i],
        "pl_lstm": pred[0],  # divide pl by 100
        "pd_lstm": pred[1],
        "pdv_lstm": pred[2]
    })

     # Update the sequence
    pred_scaled_reshaped = pred_scaled.reshape(1, -1)  # shape: (1, 3)
    last_sequence = np.vstack((last_sequence[1:], pred_scaled_reshaped))

# --- Save to CSV ---
result_df = pd.DataFrame(results)
output_file = 'lstm_result.csv'
result_df.to_csv(output_file, index=False)

print(f"Forecast results saved to {output_file}")

Forecast results saved to lstm_result.csv


In [11]:
result = pd.read_csv('lstm_result.csv')

In [12]:
result.head()

,PERIOD_START_TIME,pl_lstm,pd_lstm,pdv_lstm
0,2022-09-27 06:00:00,0.008045,0.732821,0.038827
1,2022-09-27 06:15:00,0.009546,0.733214,0.041284
2,2022-09-27 06:30:00,0.011018,0.733203,0.043763
3,2022-09-27 06:45:00,0.012470,0.732366,0.046232
4,2022-09-27 07:00:00,0.014033,0.733021,0.048869


# XGBOOST

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [14]:
# Load the data
df = pd.read_csv("/kaggle/input/forecastingdata/Data1.csv")
df['PERIOD_START_TIME'] = pd.to_datetime(df['PERIOD_START_TIME'])

# Filter only between 6AM and midnight
df = df[(df['PERIOD_START_TIME'].dt.hour >= 6) & (df['PERIOD_START_TIME'].dt.hour < 24)]


In [15]:
lag_df = df.copy()
for lag in range(1, 4):
    for col in ['pl', 'pd', 'pdv']:
        lag_df[f'{col}_lag{lag}'] = lag_df[col].shift(lag)

lag_df.dropna(inplace=True)

In [16]:
# Define test date
test_date = pd.to_datetime("2022-09-27").date()

# Split the lagged dataframe
train_lag = lag_df[lag_df['PERIOD_START_TIME'].dt.date < test_date]
test_lag = lag_df[lag_df['PERIOD_START_TIME'].dt.date == test_date]


In [17]:
# ---- 3. Predict with XGBoost ----
result_dict = {'PERIOD_START_TIME': test_lag['PERIOD_START_TIME'].values}
# Dictionary to store scalers for inverse transformation
scalers_xgb = {}

for col in ['pl', 'pd', 'pdv']:
    X_train = train_lag[[f'{col}_lag1', f'{col}_lag2', f'{col}_lag3']]
    y_train = train_lag[col].values.reshape(-1, 1) # Reshape for scaler

    # Create and fit a scaler for the target variable
    scaler = MinMaxScaler()
    y_train_scaled = scaler.fit_transform(y_train)
    scalers_xgb[col] = scaler

    # Prepare test data for prediction
    X_test = test_lag[[f'{col}_lag1', f'{col}_lag2', f'{col}_lag3']]

    # Initialize and train the XGBoost model
    model = XGBRegressor(
        objective='reg:tweedie',
        tweedie_variance_power=1.5,
        learning_rate=0.1,
        max_depth=4,
        n_estimators=100,
        random_state=42
    )
    model.fit(X_train, y_train_scaled)

    # Make scaled predictions
    y_pred_scaled = model.predict(X_test).reshape(-1, 1)
    
    # Inverse transform the predictions to the original scale
    y_pred_original = scaler.inverse_transform(y_pred_scaled).flatten()
    
    # Clip any negative values that might occur
    y_pred_original = np.clip(y_pred_original, 0, None)

    result_dict[f'{col}_xgb'] = y_pred_original

# --- Save to CSV (your original code) ---
result_df = pd.DataFrame(result_dict)
output_file = 'xgb_result.csv'
result_df.to_csv(output_file, index=False)
print(f"Forecast results saved to {output_file}")

Forecast results saved to xgb_result.csv


In [18]:
xgb = pd.read_csv('/kaggle/working/xgb_result.csv')

In [19]:
xgb.head()

,PERIOD_START_TIME,pl_xgb,pd_xgb,pdv_xgb
0,2022-09-27 06:00:00,0.005397,0.729268,0.031656
1,2022-09-27 06:15:00,0.005340,0.718475,0.025296
2,2022-09-27 06:30:00,0.002834,0.729042,0.025536
3,2022-09-27 06:45:00,0.001999,0.719361,0.025715
4,2022-09-27 07:00:00,0.001527,0.719169,0.024920


# Side by Side Comparision

In [20]:
df = pd.read_csv("/kaggle/input/forecastingdata/Data1.csv")
lstm_df = pd.read_csv("/kaggle/working/lstm_result.csv")
xgb_df = pd.read_csv("/kaggle/working/xgb_result.csv")

In [21]:
df['PERIOD_START_TIME'] = pd.to_datetime(df['PERIOD_START_TIME'])
original_25sep = df[df['PERIOD_START_TIME'].dt.date == pd.to_datetime("2022-09-27").date()]
original_25sep = original_25sep[['PERIOD_START_TIME', 'pl', 'pd', 'pdv']].reset_index(drop=True)


In [22]:
xgb_df['PERIOD_START_TIME'] = pd.to_datetime(xgb_df['PERIOD_START_TIME'])
lstm_df['PERIOD_START_TIME'] = pd.to_datetime(lstm_df['PERIOD_START_TIME'])


In [23]:
merged_df = original_25sep.merge(
    xgb_df, on='PERIOD_START_TIME', how='left'
).merge(
    lstm_df, left_on='PERIOD_START_TIME', right_on='PERIOD_START_TIME', how='left'
)

In [24]:
final = merged_df[[
    'PERIOD_START_TIME',
    'pl', 'pl_lstm','pl_xgb',
    'pd','pd_lstm', 'pd_xgb', 
    'pdv', 'pdv_lstm', 'pdv_xgb'
]]

In [25]:
final_df = pd.DataFrame(final)
output = 'final_result.csv'
final_df.to_csv(output, index=False)

print(f"Forecast results saved to {output}")

Forecast results saved to final_result.csv


In [26]:
result = pd.read_csv('final_result.csv')

In [27]:
result.head()

,PERIOD_START_TIME,pl,pl_lstm,pl_xgb,pd,pd_lstm,pd_xgb,pdv,pdv_lstm,pdv_xgb
0,2022-09-27 06:00:00,0.001182,0.008045,0.005397,0.716064,0.732821,0.729268,0.022242,0.038827,0.031656
1,2022-09-27 06:15:00,0.001182,0.009546,0.005340,0.723046,0.733214,0.718475,0.022997,0.041284,0.025296
2,2022-09-27 06:30:00,0.000939,0.011018,0.002834,0.718230,0.733203,0.729042,0.022982,0.043763,0.025536
3,2022-09-27 06:45:00,0.000719,0.012470,0.001999,0.712220,0.732366,0.719361,0.022649,0.046232,0.025715
4,2022-09-27 07:00:00,0.000765,0.014033,0.001527,0.721189,0.733021,0.719169,0.024296,0.048869,0.024920
